# Multi-Class CNN untuk Deteksi Serangan Siber

Notebook ini mendemonstrasikan cara melatih Convolutional Neural Network (CNN) untuk mengklasifikasikan berbagai jenis serangan siber.
Kita akan melalui langkah-langkah berikut:
1. **Memuat dan Eksplorasi Data**: Membaca dataset dan memahami strukturnya.
2. **Visualisasi Data Awal**: Melihat distribusi kelas sebelum diproses.
3. **Preprocessing Data**: Mengubah label menjadi angka, penskalaan fitur numerik, dan menangani variabel kategorikal.
4. **Menangani Data Tidak Seimbang (Imbalanced Data)**: Menggunakan SMOTE dan RandomOverSampler untuk menyeimbangkan dataset.
5. **Persiapan Data untuk CNN**: Mengubah bentuk data (reshape) ke format 3D yang dibutuhkan oleh CNN.
6. **Membangun Model CNN**: Membuat arsitektur CNN 1D.
7. **Melatih Model**: Melatih model dengan callbacks untuk performa optimal.
8. **Evaluasi dan Visualisasi**: Memeriksa akurasi, confusion matrix, dan kurva pembelajaran.
9. **Menyimpan Model**: Menyimpan model yang telah dilatih.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline as ImbPipeline
import warnings
warnings.filterwarnings('ignore')

# Set style untuk plot
sns.set(style="whitegrid")

## 1. Memuat Data
Kita memuat dataset dari file CSV `cyberfeddefender_dataset.csv`.

In [ ]:
df = pd.read_csv('cyberfeddefender_dataset.csv')
print("Ukuran Dataset:", df.shape)
df.head()

## 2. Mapping Label & Visualisasi Awal
Kita mengubah label string ('Normal', 'DDoS', 'Ransomware', 'Brute Force') menjadi nilai numerik (0, 1, 2, 3) agar dapat diproses oleh model.
Selain itu, kita akan memvisualisasikan distribusi kelas untuk melihat apakah data seimbang atau tidak.

In [ ]:
label_mapping = {'Normal': 0, 'DDoS': 1, 'Ransomware': 2, 'Brute Force': 3}
df['True_Label'] = df['Attack_Type'].map(label_mapping)
class_names = ['Normal', 'DDoS', 'Ransomware', 'Brute Force']

print("Distribusi kelas sebelum oversampling:")
print(df['True_Label'].value_counts().sort_index())

In [ ]:
# Visualisasi Distribusi Kelas Awal
plt.figure(figsize=(8, 5))
ax = sns.countplot(x='True_Label', data=df, palette='viridis')
ax.set_xticklabels(class_names)
plt.title('Distribusi Kelas Sebelum Oversampling', fontsize=14)
plt.xlabel('Tipe Serangan')
plt.ylabel('Jumlah Sampel')
plt.show()

## 3. Preprocessing & Oversampling
Kita memisahkan fitur (X) dan target (y). Kita juga mendefinisikan kolom mana yang numerik dan mana yang kategorikal.
Kita menggunakan `ColumnTransformer` untuk menerapkan:
- `StandardScaler` pada fitur numerik (untuk normalisasi).
- `OneHotEncoder` pada fitur kategorikal (untuk mengubahnya menjadi format numerik).

Kemudian kita menggunakan `SMOTE` dan `RandomOverSampler` untuk menyeimbangkan kelas, memastikan model tidak bias terhadap kelas mayoritas.

In [ ]:
# Memisahkan fitur dan target
X = df.drop(columns=['Label', 'Attack_Type', 'Timestamp', 'Source_IP', 'Destination_IP'])
y = df['True_Label']

# Definisi fitur
numerical_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_features = ['Protocol', 'Flags']

# Membuat preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
    ],
    remainder='passthrough'
)

# Definisi pipeline oversampling
smote = SMOTE(sampling_strategy='not majority', k_neighbors=5, random_state=42)
ros = RandomOverSampler(sampling_strategy='all', random_state=42)

balancing_pipeline = ImbPipeline([
    ('preprocessor', preprocessor),
    ('smote', smote),
    ('ros', ros)
])

print("Sedang melakukan oversampling...")
X_balanced, y_balanced = balancing_pipeline.fit_resample(X, y)

print("\nDistribusi kelas SETELAH oversampling:")
for i in range(4):
    count = np.sum(y_balanced == i)
    print(f"  {i} - {class_names[i]:12} -> {count} sampel")
print(f"Total sampel: {len(X_balanced)}")

In [ ]:
# Visualisasi Distribusi Kelas Setelah Oversampling
plt.figure(figsize=(8, 5))
unique, counts = np.unique(y_balanced, return_counts=True)
ax = sns.barplot(x=unique, y=counts, palette='viridis')
ax.set_xticklabels(class_names)
plt.title('Distribusi Kelas Setelah Oversampling', fontsize=14)
plt.xlabel('Tipe Serangan')
plt.ylabel('Jumlah Sampel')
plt.show()

## 4. Split Data dan Reshape
Kita membagi data menjadi set pelatihan dan pengujian (70% latih, 30% uji).
CNN mengharapkan input dalam format 3D: `(samples, time_steps, features)`. Karena kita memiliki data tabular 1D, kita mengubah bentuknya menjadi `(samples, features, 1)`.

In [ ]:
# One-hot encode target
y_cat = to_categorical(y_balanced, num_classes=4)

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X_balanced, y_cat, test_size=0.3, random_state=42, stratify=y_balanced
)

# Reshape untuk CNN 1D
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

print(f"Ukuran Train: {X_train.shape}")
print(f"Ukuran Test: {X_test.shape}")

## 5. Membangun Model CNN
Kita membangun Convolutional Neural Network 1D.
- **Conv1D**: Mengekstrak fitur dari input.
- **BatchNormalization**: Menstabilkan pembelajaran.
- **MaxPooling1D**: Mengurangi dimensi data.
- **Dropout**: Mencegah overfitting.
- **Dense**: Layer fully connected untuk klasifikasi akhir.

In [ ]:
model = Sequential([
    Conv1D(64, 3, activation='relu', input_shape=(X_train.shape[1], 1), padding='same'),
    BatchNormalization(),
    Conv1D(64, 3, activation='relu', padding='same'),
    MaxPooling1D(2),
    Dropout(0.3),

    Conv1D(128, 3, activation='relu', padding='same'),
    BatchNormalization(),
    Conv1D(128, 3, activation='relu', padding='same'),
    MaxPooling1D(2),
    Dropout(0.4),

    Conv1D(256, 3, activation='relu', padding='same'),
    BatchNormalization(),
    MaxPooling1D(2),
    Dropout(0.5),

    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

## 6. Melatih Model
Kita melatih model menggunakan:
- **EarlyStopping**: Menghentikan pelatihan jika akurasi validasi tidak meningkat selama 3 epoch.
- **ReduceLROnPlateau**: Mengurangi learning rate jika loss validasi stagnan.

In [ ]:
early_stop = EarlyStopping(
    monitor='val_accuracy', patience=3, restore_best_weights=True, mode='max', verbose=1
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=8, min_lr=1e-7, verbose=1
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=10,
    batch_size=16,
    callbacks=[early_stop, reduce_lr],
    verbose=1
)

## 7. Evaluasi & Visualisasi
Kita mengevaluasi model pada data uji dan menampilkan laporan klasifikasi serta visualisasi performa.

In [ ]:
y_pred = np.argmax(model.predict(X_test), axis=1)
y_true = np.argmax(y_test, axis=1)
acc = accuracy_score(y_true, y_pred)

print("="*80)
print(f"AKURASI FINAL: {acc:.5f} -> {acc*100:.3f}%")
print("="*80)
print("\nLaporan Klasifikasi:")
print(classification_report(y_true, y_pred, target_names=class_names, digits=4))

### Confusion Matrix
Visualisasi confusion matrix untuk melihat seberapa baik model memprediksi setiap kelas.

In [ ]:
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(9,7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False,
            xticklabels=class_names, yticklabels=class_names)
plt.title(f'Confusion Matrix - Akurasi: {acc*100:.3f}%', fontsize=16)
plt.ylabel('Label Asli', fontsize=14)
plt.xlabel('Label Prediksi', fontsize=14)
plt.tight_layout()
plt.show()

### Kurva Pembelajaran (Learning Curves)
Plot akurasi dan loss selama epoch untuk memeriksa overfitting atau underfitting.

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1,2,1)
plt.plot(history.history['accuracy'], label='Akurasi Train')
plt.plot(history.history['val_accuracy'], label='Akurasi Validasi')
plt.title('Akurasi Model')
plt.xlabel('Epoch'); plt.ylabel('Akurasi')
plt.legend(); plt.grid(True)

plt.subplot(1,2,2)
plt.plot(history.history['loss'], label='Loss Train')
plt.plot(history.history['val_loss'], label='Loss Validasi')
plt.title('Loss Model')
plt.xlabel('Epoch'); plt.ylabel('Loss')
plt.legend(); plt.grid(True)
plt.tight_layout()
plt.show()

## 8. Menyimpan Model
Kita menyimpan model yang telah dilatih ke file (`cnn_multiclass_model.h5`).
Jika Anda menggunakan Google Colab, Anda dapat mengunduhnya menggunakan kode di bawah ini.

In [ ]:
# Menyimpan model
model_filename = 'cnn_multiclass_model.h5'
model.save(model_filename)
print(f"Model disimpan sebagai {model_filename}")

# Kode untuk mengunduh jika berjalan di Colab (dikomentari)
# try:
#     from google.colab import files
#     files.download(model_filename)
# except ImportError:
#     print("Tidak berjalan di Colab, model disimpan secara lokal.")